In [11]:
# ==================================================================
# Script to generate jobs for image means and stdev
# ==================================================================

import os
import sys
import glob #for wildcard matching
import time
import subprocess
import nibabel as nib
import numpy as np

#Import my own library
sys.path.append('/data/chamal/projects/anthony/Analysis_CoBrA/')
import NiiImgUtils as IU

In [12]:
"""Anthony's notes
- Note that my own library ('NiiImgUtils') must be imported (as 'IU')

"""
print "All is good"

All is good


In [18]:
# ==================================================================
# Setup the directories
# ==================================================================

""" Note on directory hierarchy for the input, reference and transform files 
parent_dir
    |
    +-- subj_dir_1 (as indicated by dir_wc)
    |        |
    |        +- file_of_interest (as indicated by file_wc)
    |
    +-- subj_dir_2 ...
"""

#Temp variable for how many segs - take off in the future for generalized use
seg_num = '10' #TODO: delete for generalized use!
# ========================================
# Input related
# ========================================
in_parent_dir='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/seg'+seg_num+'_tract2voxel_probability_labels/model_space'
in_dir_wc='[0-9][0-9][0-9][0-9][0-9][0-9]'
in_file_wc='*_region_seg_pct_modelSpace.nii.gz'

# ========================================
# Output related
# ========================================
# Output file paths
mean_out_path= os.path.join(in_parent_dir,'seg'+seg_num+'_winPct_avg_modelSpace.nii.gz')
stdev_out_path=os.path.join(in_parent_dir,'seg'+seg_num+'_winPct_stdev_modelSpace.nii.gz')
# Job document paths
jobDoc_dir='/data/chamal/projects/anthony/qbatch_jobDocs'
jobList_name='autoSubmit_seg5_winPct_meanStdev_jobList.sub'
jobScript_stdev_name='autoSubmit_seg5_winPct_meanStdev_jobScript.py'

In [19]:
# ====================================================================
# Find and save the filenames into lists
# ====================================================================
#### Initilize and save input files paths
in_file_list=glob.glob(os.path.join(in_parent_dir,in_dir_wc,in_file_wc))
in_file_list.sort()


#Printout
print "File initilization stats:"
print "\tTotal input files: %d" % len(in_file_list)


File initilization stats:
	Total input files: 100


In [7]:
IU.multiSubj_stdev_iterative(in_file_list[0:3],mean_out_path, stdev_out_path)

Opening mean image...
Initializing & processing first image...
Processing image data, subject 2 out of 3; 

TypeError: 'numpy.ndarray' object is not callable

In [9]:
# ====================================================================
# Generate job items for stdev calculation
# ====================================================================
SUBMIT = True
# Qbatch specs (if auto-submit is True above)
WALLTIME = '05:00:00'
CHUNKSIZE = '1' #Default = 1, keep at 1 if possible
PROCESSOR_PER_JOB = '14' #Increase both processing power (if multithreading possible) and RAM (1.75GB per processor)


# Path to the directory containing the library that calculates stdev (written by Anthony)
stdev_calculation_lib_dir = "/data/chamal/projects/anthony/Analysis_CoBrA/"

####### Generate the python script to be called ########
jobScript_path = os.path.join(jobDoc_dir, jobScript_stdev_name)
with open(jobScript_path,'w') as script_outStream:
    #Import modules
    script_outStream.write("import sys \n")
    script_outStream.write("sys.path.append('%s') \n" % stdev_calculation_lib_dir) 
    script_outStream.write("import NiiImgUtils as IU \n")
    #Append the list of subject file paths
    script_outStream.write("\nsubj_list = []\n")
    for subj_file in in_file_list:
        script_outStream.write("subj_list.append('%s')\n" % subj_file)
    #Call the function for mean and standard deviation
    script_outStream.write("\nIU.multiSubj_stdev_wholeBrain(subj_list,'%s')\n"%stdev_out_path)

####### Generate the command that calls the python script ########
jobList_path = os.path.join(jobDoc_dir, jobList_name)
with open(jobList_path,'w') as jobList_outStream:
    jobList_outStream.write("python %s" % jobScript_path)


####### Submit the job if indicated ########
if SUBMIT == True:
    cmd = ['qbatch','-w',WALLTIME, '-c',CHUNKSIZE, '--ppj', PROCESSOR_PER_JOB, jobList_path]
    subprocess.call(cmd)


In [21]:
# ====================================================================
# Calculate the voxel-wise mean as it is faster
# ====================================================================

IU.multiSubj_mean(in_file_list,mean_out_path)

Initializing subject 1 / reference subject...
Processing image data, subject 100 out of 100; 
Calculating overall average...
Writing to output file...
Done!
